In [86]:
import os, sys
import numpy as np
import pandas as pd
 
from sklearn.ensemble import RandomForestClassifier

# Load Data

Read the data into a panda dataframe

Build training data using the not null data and a prediction set using the null data.

Then split up the training data into a training set and a cross validation set by randomly splitting items up by their program priority code.

In [42]:
df = pd.read_excel('la_data.xlsx')

In [112]:
full_data = df[df["Expense_Type"].notnull()]
full_data = full_data.dropna(how='any')
empty_data = df[False == df["Expense_Type"].notnull()]


single_empty = empty_data[empty_data["Program_Name"].notnull()]
all_empty = empty_data[False == empty_data["Program_Name"].notnull()]

priorities = full_data["Program_Priority"].unique()
programs = full_data["Program_Name"].unique()
expenses = full_data["Expense_Type"].unique()
departments = full_data["Dept_Code"].unique()

# Convert Strings to Ints

Before trying to run word2vec, try just running random forest on data giving each string a unique int id

In [113]:

def string_col_to_int(column, keys):
    
    for i, key in enumerate(keys):
        
        try:
        
            index = full_data[column] == key
        except:
            return
        
        full_data.loc[index, column] = i
        
        
string_col_to_int("Program_Priority", priorities)
string_col_to_int("Expense_Type", expenses)
string_col_to_int("Program_Name", programs)
string_col_to_int("Dept_Code", departments)



Other than Dept_Code and Program name, there don't appear to be any great correlations with Expense type. I'm not too optimistic about this approach, but let's see how it goes.

In [114]:
full_data.corr()

,Dept_Code,Program_Name,Program_Priority,Appropriation,Fiscal_Year,Expense_Type
Dept_Code,1.000000,0.543935,-0.179315,0.028250,0.135587,0.292920
Program_Name,0.543935,1.000000,-0.226268,0.054728,0.222927,0.353792
Program_Priority,-0.179315,-0.226268,1.000000,0.062527,-0.284997,-0.048432
Appropriation,0.028250,0.054728,0.062527,1.000000,0.017380,0.094468
Fiscal_Year,0.135587,0.222927,-0.284997,0.017380,1.000000,0.146356
Expense_Type,0.292920,0.353792,-0.048432,0.094468,0.146356,1.000000


In [183]:
guaranteed = ["Dept_Code", "Appropriation"]
not_guaranteed = ["Program_Name", "Program_Priority"]
all_keys = guaranteed + not_guaranteed


def train_and_score_rfc(data, train_keys,  iterations=1):
    
    train, cv = random_cv_split(full_data)
    
    rfc_low = RandomForestClassifier(n_estimators=10)

    
    for i in range(iterations):
        rfc_low.fit(train[train_keys], train["Expense_Type"])
        score = rfc_low.score(cv[train_keys], cv["Expense_Type"])
        print("Keys: ", train_keys)
        print(" Acc: ", score )

    
    return rfc_low
    
rfc = train_and_score_rfc(full_data, ["Dept_Code"])
rfc = train_and_score_rfc(full_data, ["Appropriation"])
rfc = train_and_score_rfc(full_data, ["Program_Name"])
rfc = train_and_score_rfc(full_data, ["Program_Priority"])
rfc = train_and_score_rfc(full_data, guaranteed)
rfc = train_and_score_rfc(full_data, not_guaranteed)

Keys:  ['Dept_Code']
 Acc:  0.571494893222
Keys:  ['Appropriation']
 Acc:  0.644586894587
Keys:  ['Program_Name']
 Acc:  0.588126588127
Keys:  ['Program_Priority']
 Acc:  0.483540076336
Keys:  ['Dept_Code', 'Appropriation']
 Acc:  0.745346062053
Keys:  ['Program_Name', 'Program_Priority']
 Acc:  0.574568659891


So interesting results here: It seems that the best individual indicator is simply Appropriation which makes sense. Then is program name and not far behind Dept code. Last is program priority which makes sense because that was a complex description. So this is kind of good news because for about 200 rows, the only info we have is the dept code and appropriation, so for those rows which we can't run embeddings on, we can still get descent results, given the ~75% accuracy of the only guaranteed keys

Now I'm going to try a few combinations just to get a better feel for data

In [184]:
rfc = train_and_score_rfc(full_data, ["Program_Name", "Appropriation"])
rfc = train_and_score_rfc(full_data, ["Program_Priority", "Appropriation"])
rfc = train_and_score_rfc(full_data, ["Program_Name", "Appropriation", "Dept_Code"] )
rfc = train_and_score_rfc(full_data, all_keys)

Keys:  ['Program_Name', 'Appropriation']
 Acc:  0.778542606219
Keys:  ['Program_Priority', 'Appropriation']
 Acc:  0.612324672338
Keys:  ['Program_Name', 'Appropriation', 'Dept_Code']
 Acc:  0.783149171271
Keys:  ['Dept_Code', 'Appropriation', 'Program_Name', 'Program_Priority']
 Acc:  0.751053864169


So the best combination is essentially all the keys minus program priority which is something to keep in mind for later. I'm interested to see how the random forest results differ with embedding as opposed to unique values.

Random Forest was my first instinct to try on this data but I'm gonna try an SVM just in case it is closer to being linearly separablee

In [185]:
from sklearn import svm


def train_and_score_svc(data, iterations=1):
    
    train, cv = random_cv_split(full_data)
    
    svc_g = svm.SVC()
    svc_ng = svm.SVC()
    
    for i in range(iterations):

        svc_g.fit(train[guaranteed], train["Expense_Type"])
        score = svc_g.score(cv[guaranteed], cv["Expense_Type"])
        print("SVC guaranteed data Score: ", score)


        svc_ng.fit(train[guaranteed+not_guaranteed], train["Expense_Type"])
        score = svc_ng.score(cv[guaranteed+not_guaranteed], cv["Expense_Type"])
        print("SVC not guaranteed data Score: ", score)
    
    
    return svc_g, svc_ng

svc_g, svc_ng = train_and_score_svc(full_data)


SVC guaranteed data Score:  0.702265372168
SVC not guaranteed data Score:  0.701571890892


Not as good results as Random Forest, but it might scale better. With a smaller dataset I might use svm, but becasue we have around 80% of the data already filled and 20% not filled, I'm leaning towards the random forest

Also, SVM performs worse given the program name and ID, because it probably just adds unneccessary complexity towards the fitting.